![raining](4.png)

In [10]:
import pandas as pd

# Функция, вытаскивающая среднегодовую температуру по странам

In [2]:
def extract_average_annual_temp(df):
    '''Функция, вытаскивающая среднегодовую температуру по странам
    
    Аргументы:
    df - Dateфрейм, в котором обязательно есть столбцы 'Date' с типом datetime, 'Код страны' и 'Cредняя температура'
    
    Возвращает:
    data - pandas.DataFrame объект со столбцами 'Country','Year',"Avg temperature"
    '''
    
    df['Year'] = df['Date'].dt.year
    data = df.groupby(['Country','Year'], as_index=False)["Avg temperature"].mean()
    data.columns = ['Country','Year',"Avg temperature"]
    return data

# Функция получения данных о погоде

Коды стран и номера метеостанций можно найти на сайте http://pogoda-service.ru/

In [16]:
def get_weather_data(country_list=['RS','US','AS','AR','CH','CA'],station_list=['27612','722950','949260','875850','545110','710630']):
    '''Функция получения данных о погоде
    
    Аргументы:
    country_list - список стран в виде их кода
    station_list - список станций в соответствии со странами в виде номеров станций
    
    Возвращает:
    data - pandas.DataFrame объект
    '''
    
    import requests

    df = []
    for j in range(len(country_list)):
        for i in range(6):
            url = r'http://pogoda-service.ru/archive_gsod_res.php?country={}&station={}&datepicker_beg=01.01.{}&datepicker_end=01.01.{}&bsubmit=Посмотреть'.format(country_list[j], station_list[j], str(2011+i),str(2012+i))
            header = {
              "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
              "X-Requested-With": "XMLHttpRequest"
            }
            r = requests.get(url, headers=header)
            df.append(pd.read_html(r.text, match=r'ÐÐ°ÑÐ°')[0])
            df[-1]['Country code'] = country_list[j]
    data = pd.concat(df, sort=False, ignore_index=True)
    df = []
    data.columns = ['Date','Max temperature','Min temperature','Avg temperature','Atmospheric pressure','Wind speed','Rainfall','Effective temperature', 'Country']
    
    data.loc[data['Country'] == 'AR', 'Country'] = 'Argentina'
    data.loc[data['Country'] == 'RS', 'Country'] = 'Russian Federation'
    data.loc[data['Country'] == 'US', 'Country'] = 'United States of America'
    data.loc[data['Country'] == 'AS', 'Country'] = 'Australia'
    data.loc[data['Country'] == 'CH', 'Country'] = 'China'
    data.loc[data['Country'] == 'CA', 'Country'] = 'Canada'
    
    data['Date'] = pd.to_datetime(data['Date'])
    return data

# Функция, вытаскивающая данные урожая

In [6]:
def get_yield_data():
    '''Функция получения данных об урожайности
    
    Возвращает:
    df - pandas.DataFrame объект со столбцами 'Country','Item','Year','Value'
    '''
    
    df = pd.read_csv('fao.csv')
    df.drop(['Domain', 'Element'], axis='columns', inplace=True)
    df.columns = ['Country','Item','Year','Value']
    df = df[df.Year >= 2011].reset_index()
    df.drop('index', axis='columns', inplace=True)
    
    df = df.assign(idx=df.groupby(['Item']).cumcount()).pivot_table(index=['Year','Country'],columns='Item', values='Value')
    df = df.reset_index(0).reset_index()
    
    return df
    

В качестве мест наблюдения возьмем столицы стран:

In [7]:
country_list = ['RS','US','AS','AR','CH','CA']
station_list = ['27612','725033','949260','875850','545110','710630']

Посмотрим на эти данные:

In [20]:
df = get_weather_data()

In [21]:
df1 = extract_average_annual_temp(df)

In [22]:
df2 = get_yield_data()

In [23]:
df

,Date,Max temperature,Min temperature,Avg temperature,Atmospheric pressure,Wind speed,Rainfall,Effective temperature,Country,Year
0,2011-01-01,15.6,3.9,10.5,1010.3,3.0,0.0,7.3,United States of America,2011
1,2011-02-01,15.6,7.8,10.3,1009.2,3.0,0.0,7.3,United States of America,2011
2,2011-03-01,14.4,7.0,9.1,1005.0,4.0,15.0,6.0,United States of America,2011
3,2011-04-01,18.9,7.2,11.8,1009.2,3.0,1.0,9.1,United States of America,2011
4,2011-05-01,20.0,7.2,12.7,1010.7,3.0,0.0,9.9,United States of America,2011
...,...,...,...,...,...,...,...,...,...,...
10500,2016-12-26,-7.4,-11.5,-10.0,1025.3,4.0,1.0,-15.3,Canada,2016
10501,2016-12-27,6.2,-2.0,2.9,995.8,4.0,17.0,-1.1,Canada,2016
10502,2016-12-28,7.1,-7.2,-3.5,1005.7,4.0,1.0,-8.2,Canada,2016
10503,2016-12-29,-0.8,-9.8,-6.6,999.9,4.0,4.0,-11.5,Canada,2016


In [13]:
df1

,Country,Year,Avg temperature
0,Argentina,2011,19.835088
1,Argentina,2012,18.298365
2,Argentina,2013,18.051913
3,Argentina,2014,21.604490
4,Argentina,2015,18.500000
5,Argentina,2016,17.715342
6,Australia,2011,12.835342
7,Australia,2012,12.479837
8,Australia,2013,13.613934
9,Australia,2014,13.707397


In [14]:
df2

Item,Country,Year,Barley,"Beans, green",Maize,Oats,Soybeans,Wheat
0,Argentina,2011,36439.0,40220.0,63503.0,21158.0,26053.0,35063.0
1,Australia,2011,21719.0,50563.0,57390.0,13652.0,17136.0,20301.0
2,Russian Federation,2011,22028.0,NaN,43444.0,18153.0,13820.0,22645.0
3,United States of America,2011,37160.0,31766.0,92146.0,20547.0,28232.0,29422.0
4,Argentina,2012,30440.0,34286.0,57346.0,19057.0,22814.0,32199.0
5,Australia,2012,22109.0,50000.0,64654.0,17261.0,22598.0,21511.0
6,Russian Federation,2012,18259.0,NaN,42388.0,14120.0,12239.0,17727.0
7,United States of America,2012,35986.0,41585.0,77270.0,21944.0,26867.0,31154.0
8,Argentina,2013,39102.0,29679.0,66037.0,19673.0,25391.0,26581.0
9,Australia,2013,20502.0,61896.0,64441.0,15388.0,22336.0,17609.0
